In [1]:
import numpy as np
import csv, string
import imp

In [2]:
def getData(path):
    data = []
    with open(path) as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            u = row[1]
            data.append(u)
    data = data[1:]
    return data

In [3]:
train_data_path = './data/propernames/train/train_data.csv'
dev_data_path   = './data/propernames/dev/dev_data.csv'
test_data_path = './data/propernames/test/test_data.csv'

train_data = getData(train_data_path)
dev_data = getData(dev_data_path)
test_data = getData(test_data_path)

print len(train_data), len(dev_data), len(test_data)

23121 2893 2862


In [4]:
# ddict = set()
# lo, hi = 2, 4
# for row in train_data:
#     for j in range(lo, hi+1):
#         for i in range(len(row)-j+1):
#             ddict.add(row[i:i+j])
# for row in dev_data:
#     for j in range(lo, hi+1):
#         for i in range(len(row)-j+1):
#             ddict.add(row[i:i+j])
# dddict = list(ddict)

In [ ]:
def character_N_gram(data, n=2, dict=dddict, sset=ddict):
    train = np.zeros((len(data),len(dict)))
    for i,row in enumerate(data):
        for k in range(2,5):
            for j in range(len(row)-k+1):
                cur = row[j:j+k]
                if cur in sset:
                    train[i,dict.index(cur)] = 1
    return train

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(analyzer='char', ngram_range=(2,4), lowercase=False, stop_words='english')
count_vect.fit(train_data)
train_x = count_vect.transform(train_data)
dev_x   = count_vect.transform(dev_data)
test_x  = count_vect.transform(test_data)

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
count_vect = TfidfVectorizer(analyzer='char', ngram_range=(2,4), lowercase=False, stop_words='english')
count_vect.fit(train_data)
train_x = count_vect.transform(train_data)
dev_x   = count_vect.transform(dev_data)
test_x  = count_vect.transform(test_data)

In [5]:
train_x.shape

(23121, 87232)

In [ ]:
train_x = character_N_gram(train_data)
dev_x   = character_N_gram(dev_data)
test_x  = character_N_gram(test_data)

In [6]:
train_label_path = './data/propernames/train/train_labels.csv'
dev_label_path   = './data/propernames/dev/dev_labels.csv'

def getLabel(path, n):
    label = np.zeros(n)
    map = ["person", "place", "movie", "drug", "company"]
    with open(path) as csvfile:
        reader = csv.reader(csvfile)
        start = 0
        for row in reader:
            if start == 0: 
                start += 1
                continue
            label[start-1] = map.index(row[1])
            start += 1
    return label

In [7]:
train_y = getLabel(train_label_path, 23121)
dev_y   = getLabel(dev_label_path, 2893)

In [ ]:
from sklearn import cross_validation

all_folds = cross_validation.KFold(train_x.shape[0], n_folds = 5)
for a,b in all_folds:
    perceptron = foo.Perceptron(eta = 0.01, epochs = 5, n_class = 5)
    perceptron.train(train_x[a],train_y[a])
    pre = perceptron.predict(train_x[b])
    cnt = pre.shape[0]
    correct = 0
    for i in range(cnt):
        if pre[i] == train_y[b][i]:
            correct += 1
    print str(correct)+"/"+str(cnt)

In [8]:
foo = imp.load_source('Perceptron', '/Users/GanHong/Desktop/CS5740_NLP/Assignment1/models/Perceptron.py')

In [9]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components = 'mle')
# pca.fit(train_x.toarray())
# train_x = pca.transform(train_x.toarray())
# test_x  = pca.transform(test_x.toarray())
# dev_x   = pca.transform(dev_x.toarray())
train_x = train_x.toarray()
dev_x   = dev_x.toarray()

In [10]:
# baseline using perceptron with character 2 gram is 70% accuracy
# mix 2-4 character n-garm is 80% accuracy

perceptron = foo.Perceptron(n_class = 5, epochs = 1, eta = 0.0001, dim = 87232)
for _ in range(20):
    perceptron.train(train_x, train_y)
    res = perceptron.predict(dev_x)
    cnt = res.shape[0]
    correct = 0
    for i in range(cnt):
        if res[i] == dev_y[i]:
            correct += 1
    print float(correct) / cnt

loss: 0.63102806972
0.738333909437
loss: 0.810259071839
0.815416522641
loss: 0.863154707841
0.819564465952
loss: 0.893516716405
0.825095057034
loss: 0.91116301198
0.825440718977
loss: 0.92612776264
0.830625648116
loss: 0.935210414774
0.842723816108
loss: 0.943125297349
0.84168683028


KeyboardInterrupt: 

In [11]:
test_x = test_x.toarray()
out = perceptron.predict(test_x)
ret = []
map = ["person", "place", "movie", "drug", "company"]
for i,k in enumerate(out):
    ret.append(map[int(k)])
np.savetxt('test1.csv', ret, fmt="%s")

In [ ]:
np.savetxt('test1.csv', range(out.shape[0]), fmt="%i")

In [ ]:
maxEnt = imp.load_source('MaxEnt', '/Users/GanHong/Desktop/CS5740_NLP/Assignment1/models/MaxEnt.py')
me = maxEnt.MaxEnt(n_class = 5)
me.fit(train_x, train_y)

In [ ]:
res = me.predict(dev_x)
cnt = res.shape[0]
correct = 0
for i in range(cnt):
    print res[i], dev_y[i]
    if res[i] == dev_y[i]:
        correct += 1
print str(correct)+"/"+str(cnt)

In [ ]:
import tensorflow as tf

res = None

# Parameters
learning_rate = 0.01
training_epochs = 20
batch_size = 512
display_step = 1

# Network Parameters
# 1024 - 512  ------ 75%   100 epochs   batch-size = 1000
# 2048 - 1024 ------ 78%   150 epochs   batch-size = 1000
n_hidden_1 = 2048
n_hidden_2 = 1024
n_input = 87232
n_classes = 5
beta = 0.001

# tf Graph input
x = tf.placeholder(tf.float32, [None, 3025])
y = tf.placeholder(tf.float32, [None, 5])

def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y)
                      +beta*tf.nn.l2_loss(weights['h1'])
                      +beta*tf.nn.l2_loss(weights['h2'])
                      +beta*tf.nn.l2_loss(biases['b1'])
                      +beta*tf.nn.l2_loss(biases['b2']))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.initialize_all_variables()

train_f_y = np.zeros((train_x.shape[0],5))
for i,j in enumerate(train_y):
    train_f_y[i,int(j)] = 1

dev_f_y = np.zeros((dev_x.shape[0],5))
for i,j in enumerate(dev_y):
    dev_f_y[i,int(j)] = 1

with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(23121/batch_size)
        for _ in range(total_batch):
            # Loop over all batches
            seq = np.arange(train_x.shape[0])
            np.random.shuffle(seq)
            seq = seq[:batch_size]
        
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: train_x[seq],
                                                          y: train_f_y[seq]})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print "Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost)
                
        # Test model
        if epoch % 1 == 0:
            correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
            # Calculate accuracy
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
            print("Accuracy:", accuracy.eval({x: dev_x, y: dev_f_y}))
            
        res = sess.run(tf.argmax(pred, 1), feed_dict={x: test_x})
        
    print "Optimization Finished!"